In [0]:
%run ../01-Config/02-config

In [0]:
from pyspark.sql.functions import current_timestamp, lit, input_file_name
customer_filepath = "s3://tangirala/source-data/products.json"
df = spark.read.json(customer_filepath).withColumn(
    "ingestion_timestamp", 
    current_timestamp()
).withColumn(
    "filename", 
    input_file_name()
)
df.show()

In [0]:
#wite products data staging tables to bronze schema in snowflake

df.write \
    .format("snowflake") \
    .options(**snowflake_config) \
    .option("dbtable", 'products') \
    .option("sfDatabase", 'ecommerce_db') \
    .option("sfSchema", 'bronze') \
    .mode("append") \
    .save()


In [0]:
import datetime
archive_folder = "/dbfs/FileStore/archive-data/products_data/"
archive_filepath = archive_folder +'_'+datetime.datetime.now().strftime("%Y%m%d%H%M%s")
dbutils.fs.mv(customer_filepath, archive_filepath)
print(archive_filepath)